In [69]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
df = pd.read_csv('/content/train dataset.csv')

In [70]:
x_columns = list(df.columns)
y_column= 'FRAUD_NONFRAUD'
x_columns.remove(y_column)

timestamp_attributes = ['PWD_UPDT_TS', 'PH_NUM_UPDT_TS', 'CUST_SINCE_DT', 'TRAN_DT', 'TRAN_TS', 'ACTVY_DT']
nominal_attributes = [
                      'CARR_NAME',
                      'RGN_NAME',
                      'STATE_PRVNC_TXT',
                      'ALERT_TRGR_CD',
                      'DVC_TYPE_TXT',
                      'AUTHC_PRIM_TYPE_CD',
                      'AUTHC_SCNDRY_STAT_TXT',
                      'CUST_STATE',
                      'ACTN_CD',
                      'ACTN_INTNL_TXT',
                      'TRAN_TYPE_CD']
numeric_attributes = ['TRAN_AMT', 'ACCT_PRE_TRAN_AVAIL_BAL', 'CUST_AGE', 'OPEN_ACCT_CT', 'WF_dvc_age']

In [71]:
# Drop timestamp attributes
df = df.drop(timestamp_attributes, axis = 'columns')
dataframes = dict()

In [72]:
# remove rows with NaN
dataframes['remove_nan'] = df.copy()
dataframes['remove_nan'] = dataframes['remove_nan'].dropna()

# replace NaN with mode
dataframes['nan_with_mode'] = df.copy()
for column in nominal_attributes:
  mean_value=dataframes['nan_with_mode'][column].mode()
  dataframes['nan_with_mode'][column].fillna(value=mean_value, inplace=True)

In [73]:
# One hot encode nominal attributes
for dataframe_type in ['remove_nan', 'nan_with_mode']:
  _df = dataframes[dataframe_type]
  for column in nominal_attributes:
    encoding = pd.get_dummies(_df[column], prefix=column)
    _df = _df.drop(column, axis = 'columns')
    _df = _df.join(encoding)
  dataframes[dataframe_type] = _df

In [74]:
# Standardize and normalize numerical attributes
for dataframe_type in ['remove_nan', 'nan_with_mode']:
  _df_normalize = dataframes[dataframe_type]
  _df_standardize = dataframes[dataframe_type]
  _df_normalize[numeric_attributes] = preprocessing.normalize(_df_normalize[numeric_attributes])
  _df_standardize[numeric_attributes] = preprocessing.normalize(_df_standardize[numeric_attributes])
  dataframes[dataframe_type] = dict()
  dataframes[dataframe_type]['normalized'] = _df_normalize
  dataframes[dataframe_type]['standardized'] = _df_standardize
